In [83]:
%load_ext autoreload
%autoreload 2
from config.config import *
import numpy as np
import roboticstoolbox as rtb
import spatialmath as sm
import spatialgeometry as sg
import urchin
from urchin import URDF
from math import *
import os
import swift

urdf_path = os.path.join(os.getcwd(), PANDA_VIRTUAL)
urdf = URDF.load(urdf_path)
panda = rtb.models.Panda()
panda_virtual = rtb.Robot.URDF(urdf_path, 'panda_finger_virtual')
panda_virtual.qz = np.zeros(panda_virtual.n)
panda_virtual.qr = np.concatenate((panda.qr, np.zeros(panda_virtual.n - panda.n)))
fingers = {'open': np.array([0.04, 0.04]), 'close': np.array([0.0, 0.0])}


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/vahdan/repos/IJRR/Screw-Panda/models/franka_description/robots/franka_description/meshes/visual/link0.dae
/home/vahdan/repos/IJRR/Screw-Panda/models/franka_description/franka_description/meshes/visual/link0.dae
/home/vahdan/repos/IJRR/Screw-Panda/models/franka_description/meshes/visual/link0.dae
/home/vahdan/repos/IJRR/Screw-Panda/models/franka_description/robots/franka_description/meshes/collision/link0.stl
/home/vahdan/repos/IJRR/Screw-Panda/models/franka_description/franka_description/meshes/collision/link0.stl
/home/vahdan/repos/IJRR/Screw-Panda/models/franka_description/meshes/collision/link0.stl
/home/vahdan/repos/IJRR/Screw-Panda/models/franka_description/robots/franka_description/meshes/visual/link1.dae
/home/vahdan/repos/IJRR/Screw-Panda/models/franka_description/franka_description/meshes/visual/link1.dae
/home/vahdan/repos/IJRR/Screw-Panda/models/franka_description/meshes/visual/link

In [140]:

def get_screw_matrices(robot):
    """Compute Modern Robotics screw matrices S and B"""
    qz = np.zeros(robot.n)
    M = robot.fkine(qz).A
    
    # Get body Jacobian and convert to B-list
    Je = robot.jacobe(qz)
    Blist = np.vstack((Je[3:], Je[:3]))  # [ω, v]
    
    # Compute S-list from B-list using adjoint
    R = M[:3, :3]
    p = M[:3, 3]
    
    AdM = np.zeros((6, 6))
    AdM[:3, :3] = R
    AdM[3:, 3:] = R
    AdM[3:, :3] = skew(p) @ R
    
    Slist = np.zeros((6, robot.n))
    for i in range(robot.n):
        Slist[:, i] = AdM @ Blist[:, i]
        
    return Slist, Blist, M

def skew(v):
    return np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])

In [131]:
Ti = panda_virtual.links[1].isrevolute

Ti

True

In [141]:
Slist, Blist, M = get_screw_matrices(panda_virtual)


print('Slist:\n', np.round(Slist, 4))
print('Blist:\n', np.round(Blist, 4))
print('M:\n', np.round(M, 5))

IndexError: index 8 is out of bounds for axis 1 with size 8

In [139]:
qz = panda_virtual.qz
# Get jacobians
J0 = panda_virtual.jacob0(qz)
Je = panda_virtual.jacobe(qz)

# Convert to Modern Robotics format
Slist = np.vstack((J0[3:], J0[:3]))  # [ω, v]
Blist = np.vstack((Je[3:], Je[:3]))  # [ω, v]

# Get home configuration 
M = panda_virtual.fkine(qz).A

print('Slist:\n', np.round(Slist, 4))
print('Blist:\n', np.round(Blist, 4))
print('M:\n', np.round(M, 5))


Slist:
 [[-0.      0.      0.      0.      0.      0.      0.      0.7071]
 [ 0.      1.      0.     -1.      0.     -1.     -0.     -0.7071]
 [ 1.      0.      1.      0.      1.      0.     -1.      0.    ]
 [ 0.      0.4896  0.     -0.1736  0.      0.2104  0.      0.    ]
 [ 0.088   0.      0.088   0.      0.088   0.      0.      0.    ]
 [ 0.     -0.088  -0.      0.0055 -0.      0.088   0.      0.    ]]
Blist:
 [[-0.      0.7071 -0.     -0.7071 -0.     -0.7071  0.      0.    ]
 [ 1.      0.      1.     -0.      1.     -0.     -1.      0.    ]
 [ 0.     -0.7071  0.      0.7071  0.      0.7071  0.      1.    ]
 [ 0.0622  0.3462  0.0622 -0.1228  0.0622  0.1488 -0.      0.    ]
 [ 0.     -0.088   0.      0.0055  0.      0.088   0.      0.    ]
 [-0.0622  0.3462 -0.0622 -0.1228 -0.0622  0.1488  0.      0.    ]]
M:
 [[ 0.70711  0.       0.70711  0.088  ]
 [ 0.70711  0.      -0.70711 -0.     ]
 [-0.       1.       0.       0.8226 ]
 [ 0.       0.       0.       1.     ]]


In [86]:
dt = 1/1000
t = np.arange(0, 3, dt)
env = swift.Swift()
env.launch(realtime=True, browser='notebook')
traj = rtb.jtraj(panda.qz, panda.qr, t)
env.add(panda_virtual)

for q in traj.q:
    panda_virtual.q = np.concatenate((q, fingers['close']))
    env.step(dt)

In [ ]:
panda_virtual.q

array([ 0.        , -0.3       ,  0.        , -2.2       ,  0.        ,
        2.        ,  0.78539816,  0.        ,  0.        ])

In [24]:
bTe

   1         0         0         0.11      
   0         0        -1         0         
   0         1         0         0         
   0         0         0         1         


In [30]:

env = swift.Swift()
env.launch(realtime=True, browser='notebook')
# Make a panda model and set its joint angles to the ready joint configuration
panda = rtb.models.Panda()
frankie = rtb.models.Frankie()
box = sg.Cuboid(scale=[0.1, 0.07, 0.03],
                color='blue')
box.T = sm.SE3(0.7, 0, 0.015)
box.set_alpha(0.5)

panda.q = panda.qr

# Set a desired and effector pose an an offset from the current end-effector pose
bTe = sm.SE3(-box.scale[0] + 0.01, 0, 0) * sm.SE3.Rx(pi)
Tep = box.T * bTe
print(box.T)
print(Tep)
# Add the robot to the simulator
env.add(panda)
env.add(box)



# Simulate the robot while it has not arrived at the goal
arrived = False
while not arrived:

    # Work out the required end-effector velocity to go towards the goal
    v, arrived = rtb.p_servo(panda.fkine(panda.q), Tep, 1)
    
    # Set the Panda's joint velocities
    panda.qd = np.linalg.pinv(panda.jacobe(panda.q)) @ v
    
    # Step the simulator by 50 milliseconds
    env.step(0.05)


[[1.    0.    0.    0.7  ]
 [0.    1.    0.    0.   ]
 [0.    0.    1.    0.015]
 [0.    0.    0.    1.   ]]
[[ 1.0000000e+00  0.0000000e+00  0.0000000e+00  6.1000000e-01]
 [ 0.0000000e+00 -1.0000000e+00 -1.2246468e-16  0.0000000e+00]
 [ 0.0000000e+00  1.2246468e-16 -1.0000000e+00  1.5000000e-02]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]


In [ ]:
import genesis as gs

gs.init(backend=gs.cuda)

scene = gs.Scene(
    show_viewer = True,
    viewer_options = gs.options.ViewerOptions(
        res           = (1280, 960),
        camera_pos    = (3.5, 0.0, 2.5),
        camera_lookat = (0.0, 0.0, 0.5),
        camera_fov    = 40,
        max_FPS       = 60,
    ),
    vis_options = gs.options.VisOptions(
        show_world_frame = True,
        world_frame_size = 1.0,
        show_link_frame  = False,
        show_cameras     = False,
        plane_reflection = True,
        ambient_light    = (0.1, 0.1, 0.1),
    ),
    renderer=gs.renderers.Rasterizer(),
)

plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka = scene.add_entity(
    gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0, 0, 0.5),
    fov    = 30,
    GUI    = False,
)

scene.build()

# render rgb, depth, segmentation, and normal
# rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)

cam.start_recording()
import numpy as np

for i in range(120):
    scene.step()
    cam.set_pose(
        pos    = (3.0 * np.sin(i / 60), 3.0 * np.cos(i / 60), 2.5),
        lookat = (0, 0, 0.5),
    )
    cam.render()
cam.stop_recording(save_to_filename='video.mp4', fps=60)

In [ ]:
import numpy as np
import genesis as gs

########################## init ##########################
gs.init(backend=gs.cuda)

########################## create a scene ##########################
scene = gs.Scene(
    show_viewer = True,
    viewer_options = gs.options.ViewerOptions(
        res           = (1280, 960),
        camera_pos    = (3.5, 0.0, 2.5),
        camera_lookat = (0.0, 0.0, 0.5),
        camera_fov    = 40,
        max_FPS       = 60,
    ),
    vis_options = gs.options.VisOptions(
        show_world_frame = True,
        world_frame_size = 1.0,
        show_link_frame  = False,
        show_cameras     = False,
        plane_reflection = True,
        ambient_light    = (0.1, 0.1, 0.1),
    ),
    renderer=gs.renderers.Rasterizer(),
)

cam = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0, 0, 0.5),
    fov    = 30,
    GUI    = False,
)


########################## entities ##########################
plane = scene.add_entity(
    gs.morphs.Plane(),
)
cube = scene.add_entity(
    gs.morphs.Box(
        size = (0.04, 0.04, 0.04),
        pos  = (0.65, 0.0, 0.02),
    )
)
franka = scene.add_entity(
    gs.morphs.MJCF(file='xml/franka_emika_panda/panda.xml'),
)
########################## build ##########################
scene.build()

motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)

# set control gains
# Note: the following values are tuned for achieving best behavior with Franka
# Typically, each new robot would have a different set of parameters.
# Sometimes high-quality URDF or XML file would also provide this and will be parsed.
franka.set_dofs_kp(
    np.array([4500, 4500, 3500, 3500, 2000, 2000, 2000, 100, 100]),
)
franka.set_dofs_kv(
    np.array([450, 450, 350, 350, 200, 200, 200, 10, 10]),
)
franka.set_dofs_force_range(
    np.array([-87, -87, -87, -87, -12, -12, -12, -100, -100]),
    np.array([ 87,  87,  87,  87,  12,  12,  12,  100,  100]),
)

In [ ]:

# get the end-effector link
end_effector = franka.get_link('hand')

# move to pre-grasp pose
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = np.array([0.65, 0.0, 0.25]),
    quat = np.array([0, 1, 0, 0]),
)
# gripper open pos
qpos[-2:] = 0.04
path = franka.plan_path(
    qpos_goal     = qpos,
    num_waypoints = 200, # 2s duration
)
# execute the planned path
for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()
# reach
qpos = franka.inverse_kinematics(
    link = end_effector,
    pos  = np.array([0.65, 0.0, 0.130]),
    quat = np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
for i in range(100):
    scene.step()

# grasp
franka.control_dofs_position(qpos[:-2], motors_dof)
franka.control_dofs_force(np.array([-0.5, -0.5]), fingers_dof)

for i in range(100):
    scene.step()

# lift
qpos = franka.inverse_kinematics(
    link=end_effector,
    pos=np.array([0.65, 0.0, 0.28]),
    quat=np.array([0, 1, 0, 0]),
)
franka.control_dofs_position(qpos[:-2], motors_dof)
for i in range(200):
    scene.step()